<a href="https://colab.research.google.com/github/priyvantkumar9661-png/Object-ditection-with-Ai/blob/main/object_ditection_with_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics opencv-python-headless matplotlib pillow

In [ ]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from google.colab import files

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print("Uploaded:", file_name)

In [ ]:
model=YOLO('yolov8n.pt')

In [ ]:
results = model(file_name)
annotated = results[0].plot()

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(annotated,cv2.COLOR_BGR2RGB))
plt.axis('off')

In [ ]:
uploaded = files.upload()
video_path=list(uploaded.keys())[0]
print("Uploaded:",video_path)

In [ ]:
output_path ="output_detected.mp4"
cap=cv2.VideoCapture(video_path)
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps=cap.get(cv2.CAP_PROP_FPS)or 25.0
fourcc=cv2.VideoWriter_fourcc(*'mp4v')
out=cv2.VideoWriter(output_path,fourcc,fps,(width,height))

frame_count=0
while True:
  ret,frame= cap.read()
  if not ret:
    break
  results=model(frame)
  annotated=results[0].plot()
  out.write(annotated)
  frame_count+=1
  if frame_count%50==0:
      print(f"Processed {frame_count} frames")
  cap.release()
  out.release()

  print("Video Saved", output_path)
  files.download(output_path)

In [ ]:
!pip install -q ultralytics opencv-python-headless matplotlib pillow

from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from google.colab import output
from base64 import b64decode

model=YOLO('yolov8n.pt')

def take_photo(filename='snapshot.jpg'):
  js="""
  async function takePhoto() {
    const div = document.createElement('div');
    const capture = document.createElement('button');
    capture.textContent = 'Capture';
    div.appendChild(capture);
    document.body.appeendChild(div);

    const video = document.createElement('video');
    video.style.display='block';
    const stream = await navigator.mediaDevices.getUserMedia({video: true});
    document.body.appendChild(video);
    video.srcObject = stream;
    await video.play();

    // Resize window
    google.colab.output.selfIframeHeight(document.documentElement.scrollHeight, true);

    // Wait for capture button
    await new Promise((resolve)=> capture.onclick= resolve);

    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext('2d').drawImage(video,0,0);
    stream.getTracks().forEach(track => track.stop());
    const imgData = canvas.toDataURL('image/jpeg').split(',')[1];
    div.remove();
    return imgData;
  }
  takePhoto();
  """

  data=output.eval_js(js)
  with open(filename, 'wb') as f:
      f.write(b64decode(data))
  return filename

  filename = take_photo('snapshot.jpg')
  print("Capture:", filename)

  results=model(filename)
  annotated=results[0].plot()

  plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()

In [ ]:
from google.colab import files
import glob
import cv2
import matplotlib.pyplot as plt

uploaded = files.upload()

for img_name in uploaded.keys():
    print(f"Processing {img_name}...")
    results = model(img_name)
    annotated = results[0].plot()
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
!pip install ultralytics --quiet
from ultralytics import YOLO

In [ ]:
model=YOLO('yolov8n.pt')

In [ ]:
!curl -L "https://github.com/ultralytics/assets/releases/download/v0.0.0/coco128.zip" -o coco128.zip
!unzip -q coco128.zip -d dataset
!ls dataset/coco128

In [ ]:
%%writefile dataset/coco128.yaml
path: dataset/coco128
train: images/train2017
val: images/train2017

nc:80
names:
0: person
1: bicycle
2: car
3: motorcycle
4: airplane
5: bus
6: train
7: truck
8: boat
9: traffic light
10: fire hydrant
11: stop sign
12: parking meter
13: bench
14: bird
15: cat
16: dog
17: horse
18: sheep
19: cow
20: elephant
21: bear
22: zebra
23: giraffe
24: backpack
25: umbrella
26: handbag
27: tie
28: suitcase
29: frisbee
30: skis
31: snowboard
32: sports ball
33: kite
34: baseball bat
35: baseball glove
36: skateboard
37: surfboard
38: tennis racket
39: bottle
40: wine glass
41: cup
42: fork
43: knife
44: spoon
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
51: carrot
52: hot dog
53: pizza
54: donut
55: cake
56: chair
57: couch
58: potted plant
59: bed
60: dining table
61: toilet
62: tv
63: laptop
64: mouse
65: remote
66: keyboard
67: cell phone
68: microwave
69: oven
70: toaster
71: sink
72: refrigerator
73: book
74: clock
75: vase
76: scissors
77: teddy bear
78: hair drier
79: toothbrush

In [ ]:
model=YOLO("yolov8n.pt")
model.train(
    data="/content/dataset/coco128.yaml",
    epochs=3,
    imgsz=640,
    project="YOLO_Training",
    name="yolov8n_custom"
)

In [ ]:
results=model.predict("/content/dataset/coco128/images/train2017/000000000078.jpg", save=True)
results[0].show()

In [ ]:
!pip install streamlit==1.38.0 ultralytics==8.3.0 pyngrok==7.1.6 opencv-python-headless pillow pandas

In [ ]:
%%writefile app_streamlit.py
import streamlit as st
from ultralytics import YOLO
import numpy as np
from PIL import Image
import cv2, tempfile, os, glob
import pandas as pd
from pathlib import Path

st.set_page_config(page_title="Object Detection (Project)", layout="wide")
background_url = "https://i.bb.co/FqVqvBBN/web-back.png"

st.markdown(
    f"""
    <style>
   .stApp {{
       background: url("{background_url}") no-repeat center center fixed;
       background-size: cover;
   }}
   /* Sidebar styling */
   section[data-testid="stSidebar"] {{
       background: rgba(0, 0, 0, 0.3);
       backdrop-filter: blur(10px);
       border-radius: 12px;
       padding: 10px;
   }}
   /* File uploader styling */
   div[data-testid="stFileUploader"] {{
       background: rgba(0, 0, 0, 0.3);
       backdrop-filter: blur(10px);
       border-radius: 12px;
       padding: 15px;
   }}
   /* DataFrame styling */
   .stDataFrame {{
       background: rgba(255, 255, 255, 0.85);
       border-radius: 12px;
       padding: 10px;
       box-shadow: 0 4px 10px rgba(0,0,0,0.3);
   }}
   /* Dark text */
   h1, h2, h3, h4, h5, h6, p, label, span, div {{
       color: #1a1a1a !important;
       font-weight: 500;
   }}
   </style>
    """,
    unsafe_allow_html=True
)

st.title("Object Detection (Project)")

uploaded_weights = st.sidebar.file_uploader("Upload custom .pt weights (optional)", type=["pt"])
conf = st.sidebar.slider("Confidence threshold", 0.0,1.0, 0.25,0.01)
img_size = st.sidebar.selectbox("Inference image size (px)", [320,416,640,1280], index=2)

@st.cache_resourse
def load_model (weights_path="yolov8n.pt"):
  return YOLO(weights_path)

def save_uploaded_file(uploaded_file, suffix=""):
  suffix= suffix if suffix else Path(uploaded_file.name).suffix
  tf = tempfile.NameTemporaryFile(delete=False, suffix=suffix)
  tf.write(uploaded_file.getbuffer())
  tf.flush()
  return tf.name

def annotate_and_table(results, model):
  res=results[0]
  try:
    plotted = res.plot()
    annotated = cv2.cvtColor(plotted, cv2.COLOR_BGR2RGB)
  except Exception:
    annotated = res.orig_img if hasattr(res, "orig_img") else None

  detections = []
  try:
    boxes = res.boxes
    if boxes is not None and len(boxes) > 0:
           for c, cf, box in zip(boxes.cls.cpu().numpy(), boxes.conf.cpu().numpy(), boxes.xyxy.cpu().numpy()):
               name = model.names[int(c)]
               detections.append({"class": name, "conf": float(cf), "bbox": [float(x) for x in box]})
  except:
       detections = []

  return annotated, pd.DataFrame(detections)

weights_to_load = "yolov8n.pt"
if uploaded_weights:
   weights_to_load = save_uploaded_file(uploaded_weights, suffix=".pt")
   st.sidebar.success("Using uploaded weights")

model = load_model(weights_to_load)

mode = st.radio("Select input", ["Image upload", "Video upload"])

if mode == "Image upload":
   uploaded = st.file_uploader("Upload image", type=["jpg","jpeg","png"])
   if uploaded:
       img = Image.open(uploaded).convert("RGB")
       st.image(img, caption="Input image")
       results = model.predict(np.array(img), conf=conf, imgsz=img_size)
       annotated, df = annotate_and_table(results, model)
       if annotated is not None:
           st.image(annotated, caption="Annotated")
       if not df.empty:
           st.dataframe(df)

elif mode == "Video upload":
   uploaded_vid = st.file_uploader("Upload video", type=["mp4","mov","avi","mkv"])
   if uploaded_vid:
       tmp = save_uploaded_file(uploaded_vid)
       st.video(tmp)
       project_dir = tempfile.mkdtemp()
       results = model.predict(source=tmp, conf=conf, imgsz=img_size, project=project_dir, name="run", save=True)
       try:
           out_dir = str(results[0].save_dir)
           vids = glob.glob(os.path.join(out_dir, "*"))
           vids = [v for v in vids if Path(v).suffix.lower() in [".mp4",".avi",".mov",".mkv"]]
           if vids:
               st.success("Annotated video")
               st.video(vids[0])
       except:
           st.warning("Could not display annotated video")

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "32jGxH70eocxfCx43QxP6xInVoV_2cgt7tYcgWmT9AXjbVzAc" #Use your own authentication token.
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

public_url = ngrok.connect(8501)
print("Public URL:", public_url)

!streamlit run app_streamlit.py --server.port 8501 &

import time
while True:
  time.sleep(60)